<h2 style="text-align: center;">Data Mining & Grundlagen Maschinelles Lernen 1</h2>
<h3 style="text-align: center;">Wintersemester 2023/24</h3>
<h4 style="text-align: center;">Semesterprojekt: Umsatzvorhersage</h4>


# Business Understanding

Im Rahmen des Semesterprojekts für das Modul "Data Mining & Grundlagen Maschinelles Lernen 1" widmen wir uns einer praxisnahen Aufgabe – der Umsatzvorhersage für eine große Supermarktkette in Deutschland. Unser Ziel ist es, ein zuverlässiges Vorhersagemodell zu entwickeln, das auf Basis historischer Daten und verschiedener Einflussfaktoren den erwarteten Tagesumsatz jeder Filiale prognostiziert.
Eine genaue Vorhersage ermöglicht es den Filialleitungen, die Anzahl der Mitarbeiterinnen und Mitarbeiter optimal zu planen und so die Kundenzufriedenheit zu maximieren, während gleichzeitig die Personalkosten minimiert werden.

# Data Understanding

In [19]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from math import *
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, accuracy_score, r2_score
from sklearn.tree import DecisionTreeRegressor
import time
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier
from sklearn.inspection import permutation_importance
import plotly.express as px

In [23]:
# Schritt 1: Laden der Datensätze
train_df = pd.read_csv('../data/raw/dmml1_train.csv')
store = pd.read_csv('../data/raw/dmml1_stores.csv')

In [276]:
train_df.describe()

,Store ID,DayOfWeek,Sales,Customers,Open,Promo,SchoolHoliday
count,246903.000000,246903.000000,246903.000000,246903.000000,246903.000000,246903.000000,246903.000000
mean,150.568879,3.996898,5844.491894,639.519524,0.830354,0.380490,0.178763
std,86.651448,1.998008,4007.013717,472.495279,0.375322,0.485508,0.383154
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,76.000000,2.000000,3745.000000,405.000000,1.000000,0.000000,0.000000
50%,150.000000,4.000000,5719.000000,617.000000,1.000000,0.000000,0.000000
75%,226.000000,6.000000,7884.000000,833.000000,1.000000,1.000000,0.000000
max,300.000000,7.000000,38025.000000,7388.000000,1.000000,1.000000,1.000000


In [277]:
store.describe()

,Store ID,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear
count,300.000000,298.000000,190.000000,190.000000,300.000000,141.000000,141.000000
mean,150.500000,5368.590604,7.189474,2007.794737,0.470000,22.886525,2011.978723
std,86.746758,7694.152795,3.236023,9.329079,0.499933,14.235816,1.627742
min,1.000000,30.000000,1.000000,1900.000000,0.000000,1.000000,2009.000000
25%,75.750000,682.500000,4.000000,2006.000000,0.000000,10.000000,2011.000000
50%,150.500000,2270.000000,8.000000,2009.000000,0.000000,22.000000,2012.000000
75%,225.250000,7107.500000,10.000000,2012.000000,1.000000,37.000000,2013.000000
max,300.000000,58260.000000,12.000000,2015.000000,1.000000,50.000000,2015.000000


In [278]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246903 entries, 0 to 246902
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Store ID       246903 non-null  int64 
 1   DayOfWeek      246903 non-null  int64 
 2   Date           246903 non-null  object
 3   Sales          246903 non-null  int64 
 4   Customers      246903 non-null  int64 
 5   Open           246903 non-null  int64 
 6   Promo          246903 non-null  int64 
 7   StateHoliday   246903 non-null  object
 8   SchoolHoliday  246903 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 17.0+ MB


In [279]:
store.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Store ID                   300 non-null    int64  
 1   StoreType                  300 non-null    object 
 2   Assortment                 300 non-null    object 
 3   CompetitionDistance        298 non-null    float64
 4   CompetitionOpenSinceMonth  190 non-null    float64
 5   CompetitionOpenSinceYear   190 non-null    float64
 6   Promo2                     300 non-null    int64  
 7   Promo2SinceWeek            141 non-null    float64
 8   Promo2SinceYear            141 non-null    float64
 9   PromoInterval              141 non-null    object 
dtypes: float64(5), int64(2), object(3)
memory usage: 23.6+ KB


In [280]:
train_df.isna().sum()

Store ID         0
DayOfWeek        0
Date             0
Sales            0
Customers        0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
dtype: int64

In [281]:
store.isna().sum()

Store ID                       0
StoreType                      0
Assortment                     0
CompetitionDistance            2
CompetitionOpenSinceMonth    110
CompetitionOpenSinceYear     110
Promo2                         0
Promo2SinceWeek              159
Promo2SinceYear              159
PromoInterval                159
dtype: int64

# Data Preparation

In [29]:
# store['StoreType'] = store['StoreType'].map({'a': 1, 'b': 2, 'c': 3, 'd': 4})
# store['Assortment'] = store['Assortment'].map({'a': 0, 'c': 1, 'b': 2})
# store['PromoInterval']= store['PromoInterval'].map({'Jan,Apr,Jul,Oct' : 0, 'Feb,May,Aug,Nov' : 1, 'Mar,Jun,Sept,Dec' : 2})
# train_df['StateHoliday'] = train_df['StateHoliday'].map({'0': 0, 'a': 1, 'b': 2, 'c': 3})

In [24]:
train_df['StateHoliday'] = train_df['StateHoliday'].map({'0': "Kein Feiertag", 'a': "Feiertag", 'b': "Ostern", 'c': "Weihnachten"})
store['StoreType'] = store['StoreType'].map({'a': "StoreType A", 'b': "StoreTypee B", 'c': "StoreType C", 'd': "StoreType D"})
store['Assortment'] = store['Assortment'].map({'a': "Basic", 'b': "Extra", 'c': "Extended"})

In [25]:
store["CompetitionDistance"].replace(np.nan, store["CompetitionDistance"].mean(), inplace=True)

train_df['StateHoliday'] = train_df['StateHoliday'].map({'0':0, 'a':1, 'b':2, 'c':3})
train_df['Date'] = pd.to_datetime(train_df['Date'])
train_df['Year'] = train_df['Date'].dt.year
train_df['Month'] = train_df['Date'].dt.month
train_df['Day'] = train_df['Date'].dt.day
train_df['WeekOfYear'] = train_df['Date'].dt.isocalendar().week

In [26]:
train_df = pd.merge(train_df,store,how="inner",on="Store ID")
train_df.fillna(0,inplace=True)

# Exploratory Data Analysis

## Durchschnittliche Verkäufe pro Wochentag

In [52]:
px.line(train_df.groupby('Date', as_index=False).mean(numeric_only=True), y='Sales', x='Date', title='Durchschnittlicher Umsatz pro Tag')

Die Analyse der durchschnittlichen täglichen Verkäufe zeigt, dass der Beginn der Woche, insbesondere der Montag, für die Supermarktkette am umsatzstärksten ist, während die Verkaufszahlen gegen das Wochenende hin tendenziell abnehmen. Auffallend ist der deutliche Einbruch der Verkaufszahlen am Sonntag, was darauf hindeutet, dass die meisten Filialen entweder geschlossen sind oder nur sehr begrenzt operieren.

In [51]:
px.histogram(train_df.groupby('DayOfWeek', as_index=False).mean(numeric_only=True), 
    y='Sales', 
    x='DayOfWeek', 
    nbins=7, 
    color='DayOfWeek', 
    title='Durchschnittlicher Umsatz pro Wochentag')

# Predictive Modeling

In [6]:
X = train_df
y = train_df['Sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
X_train.nunique()

Store ID                       300
DayOfWeek                        7
Sales                        16789
Customers                     3170
Open                             2
Promo                            2
StateHoliday                     4
SchoolHoliday                    2
Year                             3
Month                           12
Day                             31
WeekOfYear                      52
StoreType                        4
Assortment                       3
CompetitionDistance            250
CompetitionOpenSinceMonth       13
CompetitionOpenSinceYear        21
Promo2                           2
Promo2SinceWeek                 19
Promo2SinceYear                  8
PromoInterval                    4
dtype: int64

In [289]:
colunique = X_train.nunique
cols = (X_train.nunique() < 8 )

cat_cols = cols[cols==True].index.tolist()
num_cols = cols[cols==False].index.tolist()

In [290]:
cat_cols

['DayOfWeek',
 'Open',
 'Promo',
 'StateHoliday',
 'SchoolHoliday',
 'Year',
 'Weekend',
 'Quarter',
 'Season',
 'StoreType',
 'Assortment',
 'Promo2',
 'PromoInterval']

In [291]:
# für StandardScaler
num_cols

['Store ID',
 'Month',
 'Day',
 'WeekOfYear',
 'CompetitionDistance',
 'CompetitionOpenSinceMonth',
 'CompetitionOpenSinceYear',
 'Promo2SinceWeek',
 'Promo2SinceYear']

In [17]:
# spalten zum droppen
drop_colmn = ['Sales', 'Customers', 'Date']

# für StandardScaler
ss_colmn = ['Store ID', 'Year', 'Month', 'Day', 'WeekOfYear', 'Promo2SinceWeek', 'CompetitionDistance', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2SinceWeek', 'Promo2SinceYear']
# ss_colmn = ['Store ID', 'Month', 'Day', 'WeekOfYear', 'Promo2SinceWeek']

# für One Hot Encoding
ohe_colmn = ['StateHoliday', 'StoreType', 'Assortment', 'PromoInterval']

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

# Vorverarbeitungspipeline erstellen
preprocessor = ColumnTransformer(
    transformers=[
        ('drop_cols', 'drop', drop_colmn),
        ('scale', StandardScaler(), ss_colmn),
        ('onehot', OneHotEncoder(handle_unknown='ignore'), ohe_colmn)
    ], remainder='passthrough')

# Modellpipeline erstellen
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_jobs=-1, random_state=42))
])

# Hyperparameter-Raster definieren
param_grid = {
    'regressor__n_estimators': [300],
    'regressor__max_depth': [None],
    #'regressor__min_samples_split': [2,4,8]
}

# Grid-Suche durchführen
gs = GridSearchCV(model, param_grid=param_grid, cv=3, n_jobs=-1)
gs.fit(X_train, y_train)

ValueError: 
All the 3 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Wladi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\_encode.py", line 174, in _unique_python
    uniques = sorted(uniques_set)
TypeError: '<' not supported between instances of 'str' and 'int'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\Wladi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Wladi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py", line 1152

In [39]:
print(f"Accuracy Trainingsdaten: {gs.score(X_train, y_train)}")
print(f"Accuracy Testdaten: {gs.score(X_test, y_test)}")

Accuracy Trainingsdaten: 0.9936288680298725
Accuracy Testdaten: 0.9509011104396058


In [13]:
X_train

,Store ID,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,...,Season,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
63578,78,2,3895,459,1,0,0,0,2013,11,...,4,1,1,38710.0,4.0,2013.0,0,0.0,0.0,0.0
158510,193,7,0,0,0,0,0,0,2013,1,...,1,1,0,560.0,1.0,2011.0,0,0.0,0.0,0.0
173064,212,4,7784,611,1,1,0,0,2014,11,...,4,4,0,5070.0,0.0,0.0,1,45.0,2009.0,1.0
74498,91,6,5650,685,1,0,0,0,2014,4,...,2,1,1,9720.0,8.0,2013.0,0,0.0,0.0,0.0
232183,283,4,9855,825,1,1,0,1,2015,4,...,2,4,0,190.0,9.0,2012.0,1,40.0,2012.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,146,3,4287,535,1,0,0,0,2014,6,...,3,4,1,15720.0,3.0,2010.0,0,0.0,0.0,0.0
103694,127,5,9820,673,1,1,0,0,2015,1,...,1,1,1,9070.0,4.0,2011.0,1,37.0,2009.0,0.0
131932,161,4,4159,663,1,0,0,0,2013,11,...,4,1,0,620.0,9.0,2014.0,0,0.0,0.0,0.0
146867,180,7,0,0,0,0,0,0,2015,3,...,2,1,1,15320.0,3.0,2011.0,1,14.0,2013.0,1.0
